# Database Schema

## Database Setup

```sql
CREATE DATABASE IF NOT EXISTS carpool_db;
USE carpool_db;
```

## Core Tables

### Users Table

```sql
CREATE TABLE users (
    user_id INT PRIMARY KEY AUTO_INCREMENT,
    email VARCHAR(255) NOT NULL UNIQUE,
    username VARCHAR(100) NOT NULL UNIQUE,
    password VARCHAR(255) NOT NULL,
    role ENUM('passenger', 'driver', 'both') NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    INDEX idx_email (email),
    INDEX idx_username (username)
);
```

### Passengers Table

```sql
CREATE TABLE passengers (
    passenger_id INT PRIMARY KEY AUTO_INCREMENT,
    user_id INT NOT NULL,
    name VARCHAR(255) NOT NULL,
    age INT NOT NULL,
    gender ENUM('male', 'female', 'other') NOT NULL,
    contact VARCHAR(20) NOT NULL,
    national_id VARCHAR(50) NOT NULL UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE,
    INDEX idx_user_id (user_id),
    INDEX idx_national_id (national_id),
    CHECK (age >= 18)
);
```

### Drivers Table

```sql
CREATE TABLE drivers (
    driver_id INT PRIMARY KEY AUTO_INCREMENT,
    user_id INT NOT NULL,
    name VARCHAR(255) NOT NULL,
    age INT NOT NULL,
    gender ENUM('male', 'female', 'other') NOT NULL,
    contact VARCHAR(20) NOT NULL,
    vehicle_no VARCHAR(20) NOT NULL UNIQUE,
    national_id VARCHAR(50) NOT NULL UNIQUE,
    route_from VARCHAR(255) NOT NULL,
    route_to VARCHAR(255) NOT NULL,
    route_date DATE NOT NULL,
    route_time TIME NOT NULL,
    available_seats INT DEFAULT 4,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE,
    INDEX idx_user_id (user_id),
    INDEX idx_route (route_from, route_to, route_date),
    INDEX idx_vehicle_no (vehicle_no),
    INDEX idx_national_id (national_id),
    CHECK (age >= 18),
    CHECK (available_seats >= 0)
);
```

### Bookings Table

```sql
CREATE TABLE bookings (
    booking_id INT PRIMARY KEY AUTO_INCREMENT,
    passenger_id INT NOT NULL,
    driver_id INT NOT NULL,
    route_from VARCHAR(255) NOT NULL,
    route_to VARCHAR(255) NOT NULL,
    booking_date DATE NOT NULL,
    booking_time TIME NOT NULL,
    booking_status ENUM('pending', 'confirmed', 'cancelled', 'completed') DEFAULT 'pending',
    seats_booked INT DEFAULT 1,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    FOREIGN KEY (passenger_id) REFERENCES passengers(passenger_id) ON DELETE CASCADE,
    FOREIGN KEY (driver_id) REFERENCES drivers(driver_id) ON DELETE CASCADE,
    INDEX idx_passenger (passenger_id),
    INDEX idx_driver (driver_id),
    INDEX idx_booking_date (booking_date),
    INDEX idx_status (booking_status),
    CHECK (seats_booked > 0)
);
```

### Ratings Table

```sql
CREATE TABLE ratings (
    rating_id INT PRIMARY KEY AUTO_INCREMENT,
    booking_id INT NOT NULL,
    rated_by INT NOT NULL,
    rated_user INT NOT NULL,
    rating DECIMAL(2,1) NOT NULL,
    review TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (booking_id) REFERENCES bookings(booking_id) ON DELETE CASCADE,
    FOREIGN KEY (rated_by) REFERENCES users(user_id) ON DELETE CASCADE,
    FOREIGN KEY (rated_user) REFERENCES users(user_id) ON DELETE CASCADE,
    CHECK (rating >= 1.0 AND rating <= 5.0),
    UNIQUE KEY unique_rating (booking_id, rated_by, rated_user)
);
```

---

## Schema Updates

### Modify Users Table

```sql
USE carpool_db;

ALTER TABLE users
ADD COLUMN last_login DATETIME NULL,
ADD COLUMN isActive TINYINT(1) DEFAULT 1,
ADD COLUMN updatedAt DATETIME DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP;

ALTER TABLE users
CHANGE COLUMN created_at createdAt DATETIME DEFAULT CURRENT_TIMESTAMP;

DESCRIBE users;

SELECT * FROM users;
```

---

## Query Reference

### Schema Inspection

```sql
USE carpool_db;

SHOW TABLES;

DESCRIBE users;
DESCRIBE passengers;
DESCRIBE drivers;
DESCRIBE bookings;
```

### Basic Data Retrieval

```sql
SELECT * FROM users;
SELECT * FROM passengers;
SELECT * FROM drivers;
SELECT * FROM bookings;
```

### Booking Details with Joins

```sql
SELECT 
    b.booking_id,
    b.route_from,
    b.route_to,
    b.booking_date,
    b.booking_time,
    b.booking_status,
    p.name AS passenger_name,
    p.contact AS passenger_contact,
    d.name AS driver_name,
    d.contact AS driver_contact,
    d.vehicle_no
FROM bookings b
JOIN passengers p ON b.passenger_id = p.passenger_id
JOIN drivers d ON b.driver_id = d.driver_id;
```

### Passenger Information

```sql
SELECT 
    p.passenger_id,
    p.name,
    p.age,
    p.gender,
    p.contact,
    p.national_id,
    u.email,
    u.username
FROM passengers p
JOIN users u ON p.user_id = u.user_id;
```

### Driver Information

```sql
SELECT 
    d.driver_id,
    d.name,
    d.age,
    d.gender,
    d.contact,
    d.vehicle_no,
    d.national_id,
    d.route_from,
    d.route_to,
    d.route_date,
    d.route_time,
    u.email,
    u.username
FROM drivers d
JOIN users u ON d.user_id = u.user_id;
```

### Filter by Passenger or Driver

```sql
SELECT * FROM bookings WHERE passenger_id = 1;
SELECT * FROM bookings WHERE driver_id = 1;
```

### Search Available Drivers

```sql
SELECT 
    d.*,
    u.email,
    u.username
FROM drivers d
JOIN users u ON d.user_id = u.user_id
WHERE d.route_from = 'City A' 
  AND d.route_to = 'City B'
  AND d.route_date = '2025-10-15';
```

### Analytics

```sql
SELECT 
    booking_status,
    COUNT(*) AS total_bookings
FROM bookings
GROUP BY booking_status;
```

### Record Counts

```sql
SELECT 'users' AS table_name, COUNT(*) AS record_count FROM users
UNION ALL
SELECT 'passengers', COUNT(*) FROM passengers
UNION ALL
SELECT 'drivers', COUNT(*) FROM drivers
UNION ALL
SELECT 'bookings', COUNT(*) FROM bookings;
```